In [27]:
#Import all the nessacary libraries to run the pose estimator

#Import mediapipe to be used for the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import os #Handle folder structure
import pickle #Save and oad ML model

In [28]:
#Import the model from the binary file
with open('shoulder_press.pkl', 'rb') as f:
    model = pickle.load(f)
    print("Model Loaded")
    
print(model)

Model Loaded
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])


In [29]:
#Display the results of the prediction done by the model
draw_helpers = mp.solutions.drawing_utils 
holistic_model = mp.solutions.holistic

#Use hollistics model to list down the landmarks 
for id, landmark in enumerate(holistic_model.PoseLandmark):
    print(id, " - ", landmark)

0  -  PoseLandmark.NOSE
1  -  PoseLandmark.LEFT_EYE_INNER
2  -  PoseLandmark.LEFT_EYE
3  -  PoseLandmark.LEFT_EYE_OUTER
4  -  PoseLandmark.RIGHT_EYE_INNER
5  -  PoseLandmark.RIGHT_EYE
6  -  PoseLandmark.RIGHT_EYE_OUTER
7  -  PoseLandmark.LEFT_EAR
8  -  PoseLandmark.RIGHT_EAR
9  -  PoseLandmark.MOUTH_LEFT
10  -  PoseLandmark.MOUTH_RIGHT
11  -  PoseLandmark.LEFT_SHOULDER
12  -  PoseLandmark.RIGHT_SHOULDER
13  -  PoseLandmark.LEFT_ELBOW
14  -  PoseLandmark.RIGHT_ELBOW
15  -  PoseLandmark.LEFT_WRIST
16  -  PoseLandmark.RIGHT_WRIST
17  -  PoseLandmark.LEFT_PINKY
18  -  PoseLandmark.RIGHT_PINKY
19  -  PoseLandmark.LEFT_INDEX
20  -  PoseLandmark.RIGHT_INDEX
21  -  PoseLandmark.LEFT_THUMB
22  -  PoseLandmark.RIGHT_THUMB
23  -  PoseLandmark.LEFT_HIP
24  -  PoseLandmark.RIGHT_HIP
25  -  PoseLandmark.LEFT_KNEE
26  -  PoseLandmark.RIGHT_KNEE
27  -  PoseLandmark.LEFT_ANKLE
28  -  PoseLandmark.RIGHT_ANKLE
29  -  PoseLandmark.LEFT_HEEL
30  -  PoseLandmark.RIGHT_HEEL
31  -  PoseLandmark.LEFT_FOOT_INDE

In [30]:
#Calculate angle between 3 landmark points
def calculate_pose_angle(start_point, mid_point, end_point):
    #Convert landmark coords to numpy array
#     start_point = np.array(start_point) 
#     mid_point = np.array(mid_point) 
#     end_point = np.array(end_point) 
    
    max_angle = 180.0
    
    #[0] = x, [1] = y, [2] = z
    radians = np.arctan2(end_point[1] - mid_point[1], end_point[0] - mid_point[0]) - np.arctan2(start_point[1] - mid_point[1], start_point[0] - mid_point[0])
    #Convert to an angle
    angle = np.abs(radians * max_angle / np.pi)
    
    if angle > max_angle:
        angle = 360 - angle
        
    return angle 

In [31]:
#Display the results of the prediction done by the model
draw_helpers = mp.solutions.drawing_utils 
holistic_model = mp.solutions.holistic

#Connect the test video from the device
sample_video = cv2.VideoCapture('datasets/IMG_0126.MOV')

down = None
counter = 0
rep_list = []
frame_list = []
status = None

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        ret, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if ret == False:
            print("Done")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Predict the coordinates of the landmarks (resulrs screen)
        try:
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            
            # Filter out only the upper body landmarks
            upper_body_landmarks = [pose_landmarks_array[i] for i in [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]]
            
            # Format the upper body landmarks into a numpy array for better structuring and collapse the array to 1 dimension
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in upper_body_landmarks]).flatten() 
                              if result_frame.pose_landmarks else np.zeros(12*4))
            
            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])
            
            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]  
            
            #Append correct-incorrect probabilitiesto the frame_list
            frame_list.append(pose_class_status)
            
            #Dictionary to store the coords in pixels of the landmarks
            points = {}
            for id, landmark in enumerate(result_frame.pose_landmarks.landmark):
                height, width, center = bgr_frame.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)
                points[id] = (cx, cy)

            cv2.circle(bgr_frame, points[12], 15, (255,0,0), cv2.FILLED)
            cv2.circle(bgr_frame, points[14], 15, (255,0,0), cv2.FILLED)
            cv2.circle(bgr_frame, points[16], 15, (255,0,0), cv2.FILLED)
            
            cv2.circle(bgr_frame, points[11], 15, (255,0,0), cv2.FILLED)
            cv2.circle(bgr_frame, points[13], 15, (255,0,0), cv2.FILLED)
            cv2.circle(bgr_frame, points[15], 15, (255,0,0), cv2.FILLED)            
        
            #For Right Arm
            if calculate_pose_angle(points[16], points[14], points[12]) >= 158: 
                down = True
            
            #For Left Arm
            elif calculate_pose_angle(points[15], points[13], points[11]) >= 158:
                down = True
       
            #For Right Arm
            if down and calculate_pose_angle(points[16], points[14], points[12]) <= 90: 
                down = False
                counter += 1
                rep_list.append(frame_list)
                frame_list = []
            
            #For Left Arm
            elif down and calculate_pose_angle(points[15], points[13], points[11]) <= 90:
                down = False
                counter += 1
                rep_list.append(frame_list)
                frame_list = []
            
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (600, 60), (245, 117, 16), -1)
            
            #Display the class label inside the rectangle box
            cv2.putText(bgr_frame, 'Class'
                        , (10,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract =and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Probability'
                        , (250,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (250,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Counter'
                        , (450,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(counter)
                        , (450,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass

        
        #Display the frames    
        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

#Display the overall result of the exercise            
for i in range(counter):
    x_total = 0
    y_total = 0
    total = 0
    for list in rep_list[i]:
        if list == "correct":
            x_total += 1
        else:
            y_total += 1
        total += 1
    
    avg = (x_total / total) * 100
    
    if (avg >= 70):
        feedback = "Correct"
    else:
        feedback = "Incorrect"
    
    print("The Rep No ", i + 1, " was ", feedback)
    print("Percentage of the correct average of the Rep:" ,avg)


sample_video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 